In [ ]:
!pip install ultralytics scikit-learn pyyaml

In [3]:
import os
from ultralytics import YOLO
import yaml
import shutil
from sklearn.model_selection import train_test_split
import random
import torch
from google.colab import drive

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
class YOLOTrainer:
    def __init__(self, dataset_path: str, output_path: str):
        self.dataset_path = dataset_path
        self.output_path = output_path

    def prepare_training_data(self):
        """Prepare dataset for YOLO training"""
        dataset_dir = os.path.join(self.output_path, 'dataset')
        os.makedirs(dataset_dir, exist_ok=True)

        # Create splits directories
        splits = ['train', 'val', 'test']
        for split in splits:
            os.makedirs(os.path.join(dataset_dir, split, 'images'), exist_ok=True)
            os.makedirs(os.path.join(dataset_dir, split, 'labels'), exist_ok=True)

        # Get images from each category folder
        category_folders = ['halte', 'obstacles', 'potholes', 'tactile', 'transjakarta']
        all_images = []

        for folder in category_folders:
            folder_path = os.path.join(self.dataset_path, folder, 'images')
            if os.path.exists(folder_path):
                images = [os.path.join(folder, 'images', f) for f in os.listdir(folder_path)
                         if f.endswith(('.jpg', '.jpeg', '.png', '.webp'))]
                all_images.extend(images)

        # Split dataset
        train_files, test_val = train_test_split(all_images, train_size=0.7, random_state=42)
        val_files, test_files = train_test_split(test_val, train_size=0.5, random_state=42)

        # Dictionary to map split name to files
        split_files = {
            'train': train_files,
            'val': val_files,
            'test': test_files
        }

        # Copy files to respective directories
        for split_name, files in split_files.items():
            for filepath in files:
                # Get the corresponding label path
                image_name = os.path.basename(filepath)
                label_name = os.path.splitext(image_name)[0] + '.txt'
                category = filepath.split('/')[0]

                # Source paths
                src_img = os.path.join(self.dataset_path, filepath)
                src_label = os.path.join(self.dataset_path, category, 'labels', label_name)

                # Destination paths
                dst_img = os.path.join(dataset_dir, split_name, 'images', image_name)
                dst_label = os.path.join(dataset_dir, split_name, 'labels', label_name)

                # Copy files
                if os.path.exists(src_img):
                    shutil.copy2(src_img, dst_img)
                if os.path.exists(src_label):
                    shutil.copy2(src_label, dst_label)

        print(f"Dataset split complete:")
        print(f"Train: {len(train_files)} images")
        print(f"Validation: {len(val_files)} images")
        print(f"Test: {len(test_files)} images")

        return dataset_dir

    def create_yaml_config(self, dataset_dir: str):
        """Create YAML configuration file for training"""
        # Try to find classes.txt in any of the category folders
        category_folders = ['halte', 'obstacles', 'potholes', 'tactile', 'transjakarta']
        classes = []

        for folder in category_folders:
            classes_file = os.path.join(self.dataset_path, folder, 'classes.txt')
            if os.path.exists(classes_file):
                with open(classes_file, 'r') as f:
                    classes = [line.strip() for line in f.readlines()]
                break

        if not classes:
            raise FileNotFoundError("Could not find classes.txt in any category folder")

        # Create YAML configuration
        yaml_config = {
            'path': dataset_dir,
            'train': 'train/images',
            'val': 'val/images',
            'test': 'test/images',
            'names': {i: name for i, name in enumerate(classes)}
        }

        # Save YAML file
        yaml_path = os.path.join(self.output_path, 'dataset.yaml')
        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_config, f, sort_keys=False)

        return yaml_path

    def train(self, yaml_path: str):
        """Train YOLOv8 model"""
        # Initialize model
        model = YOLO('yolov8n.pt')  # nano model

        # Start training
        results = model.train(
            data=yaml_path,
            epochs=100,           # number of epochs
            batch=16,             # batch size
            imgsz=640,           # image size
            patience=20,          # early stopping patience
            save=True,           # save checkpoints
            device='0' if torch.cuda.is_available() else 'cpu',  # use GPU if available
            project=self.output_path,
            name='training_run'
        )

        return model, results

In [5]:
# Mount Google Drive
drive.mount('/content/drive')

# Setup paths for Google Drive
dataset_path = '/content/drive/My Drive/yolo_gemini_dataset'
output_path = '/content/drive/My Drive/yolo_training_output'

Mounted at /content/drive


In [6]:
trainer = YOLOTrainer(dataset_path, output_path)

In [7]:
# Prepare dataset
print("Preparing dataset...")
dataset_dir = trainer.prepare_training_data()

Preparing dataset...
Dataset split complete:
Train: 270 images
Validation: 58 images
Test: 59 images


In [8]:
dataset_dir

'/content/drive/My Drive/yolo_training_output/dataset'

In [9]:
# Create YAML configuration
print("Creating YAML configuration...")
yaml_path = trainer.create_yaml_config(dataset_dir)

Creating YAML configuration...


In [10]:
# Start training
print("Starting training...")
model, results = trainer.train(yaml_path)

print(f"\nTraining complete! Model saved in {output_path}/training_run")

# Validate the model
print("\nValidating model...")
metrics = model.val()

print("\nValidation metrics:")
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")

Starting training...


100%|██████████| 6.25M/6.25M [00:00<00:00, 161MB/s]


Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/yolo_training_output/dataset.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/My Drive/yolo_training_output, name=training_run, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav

100%|██████████| 755k/755k [00:00<00:00, 43.6MB/s]


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all         57        159      0.719      0.325      0.366      0.233
      transjakarta_bus         17         22      0.728        0.5      0.685       0.45
                 halte         19         20      0.711        0.8      0.799      0.532
          tactile_path         26         31      0.597      0.452       0.42      0.281
            pedestrian         17         56      0.527      0.286      0.318      0.194
             crosswalk          8          8      0.651      0.125      0.294      0.172
                stairs          2          2          1          0          0          0
              obstacle          3          4          1          0          0          0
               pothole         10         16      0.538      0.438      0.411      0.236
Speed: 0.3ms preprocess, 2.8ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to /content/drive/My Drive/yolo_training_output/training_run

Training complete! Model saved in /conten

val: Scanning /content/drive/My Drive/yolo_training_output/dataset/val/labels.cache... 56 images, 2 backgrounds, 1 corrupt: 100%|██████████| 58/58 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/My Drive/yolo_training_output/dataset/val/images/obstacle_on_jakarta_sidewalk_53.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/My Drive/yolo_training_output/dataset/val/images/pothole_in_indonesia_sidewalk_36.jpg: ignoring corrupt image/label: cannot identify image file '/content/drive/My Drive/yolo_training_output/dataset/val/images/pothole_in_indonesia_sidewalk_36.jpg'



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.30it/s]


                   all         57        159       0.69      0.328      0.357      0.227
      transjakarta_bus         17         22      0.716        0.5      0.697       0.46
                 halte         19         20      0.639      0.795      0.765      0.502
          tactile_path         26         31      0.575      0.481      0.417      0.276
            pedestrian         17         56      0.486      0.287      0.307      0.189
             crosswalk          8          8      0.568      0.125      0.275      0.171
                stairs          2          2          1          0          0          0
              obstacle          3          4          1          0          0          0
               pothole         10         16      0.532      0.438      0.396      0.219
Speed: 0.3ms preprocess, 21.9ms inference, 0.0ms loss, 5.8ms postprocess per image
Results saved to /content/drive/My Drive/yolo_training_output/training_run2

Validation metrics:
mAP50: 0.357
mAP50-